# Combining Lyft Multimode Models

### <font color="red">Notice</font>

> <b><font color="blue">This notebook is intended to help you combining your internal models, not for public test data probing ! Please use it wisely.</font></b>

# The main idea

At the early beginning of this competition, we found that <font color="blue">combining solutions from different single-mode models considerably ameliorates results</font>. Furthermore, the combining process was very easy when dealing with **single-mode** models we just have to take 3 models, one for each channel, and tune their probabilities accordingly.

Things become harder when talking about **multi-mode models** where there is no obvious order between diffrent models predictions. We come accross this order absence issue by sorting the 3 channels such that the one with the highest probability will be the first one for all the models. Then, the channel with the second highest proba will be the second channel for all the models; same for the third channel. Hence, we introduce a somehow consistent order which allow to combine different models.

As the whole process was slow with **pure pandas**, we switch to **numpy** to speed up  everything.

Let's see it in practice !

**In the rest of my code, I will be using [this public dataset](https://www.kaggle.com/kneroma/lyft-best-performing-public-kernels) for demo only**

In [1]:
import pandas as pd, numpy as np
pd.options.display.max_columns=305

In [2]:
paths = [
    "../input/lyft-best-performing-public-kernels/lyft-ensembling-raster-sizes.csv", 
    "../input/lyft-best-performing-public-kernels/lyft-prediction-with-multi-mode-confidence.csv",
]
weights = [0.4, 0.6]

In [3]:
conf_cols = np.array(["conf_0", "conf_1", "conf_2"])

In [4]:
xy_cols = [[],[],[]]
for i in range(50):
    for j in range(3):
        xy_cols[j].append(f"coord_x{j}{i}")
        xy_cols[j].append(f"coord_y{j}{i}")
xy_cols[0][:10]

['coord_x00',
 'coord_y00',
 'coord_x01',
 'coord_y01',
 'coord_x02',
 'coord_y02',
 'coord_x03',
 'coord_y03',
 'coord_x04',
 'coord_y04']

In [5]:
COLUMNS = ["timestamp", "track_id"] + list(conf_cols) + xy_cols[0] + xy_cols[1] + xy_cols[2]

# Sorting

In [6]:
def sort_df(df, sort_timestamp_track_id=True):
    
    conf_orders = np.argsort(-df[conf_cols].values,1)
    XY = np.stack([df[xy_cols[0]].values,df[xy_cols[1]].values, df[xy_cols[2]].values], axis=1)
    XY = XY[np.arange(len(XY))[:, None], conf_orders]

    df2 = pd.DataFrame(columns = COLUMNS)
    df2["timestamp"] = df["timestamp"].values
    df2["track_id"] = df["track_id"].values
    df2[xy_cols[0] + xy_cols[1] + xy_cols[2]] = XY.reshape(-1,300)
    df2[conf_cols] = df[conf_cols].values[np.arange(len(df))[:, None], conf_orders]
    
    if sort_timestamp_track_id:
        df2.sort_values(["timestamp", "track_id"], inplace=True)
        df2.reset_index(inplace=True, drop=True)
    return df2

# Combining

In [7]:
%%time

df = None
for path,w in zip(paths,weights):
    print(w, path)
    temp = pd.read_csv(path)
    temp = sort_df(temp)
    temp[COLUMNS[5:]] *= w
    if df is None:
        df = temp
    else:
        df[COLUMNS[2:]] += temp[COLUMNS[2:]]
df[conf_cols] /= df[conf_cols].sum(1).values[:, None]

sample = pd.read_csv("../input/lyft-motion-prediction-autonomous-vehicles/multi_mode_sample_submission.csv")

df = sample[["timestamp", "track_id"]].merge(df, on=["timestamp", "track_id"])
sample.shape, df.shape

0.4 ../input/lyft-best-performing-public-kernels/lyft-ensembling-raster-sizes.csv
0.6 ../input/lyft-best-performing-public-kernels/lyft-prediction-with-multi-mode-confidence.csv
CPU times: user 1min 41s, sys: 52.9 s, total: 2min 34s
Wall time: 2min 35s


((71122, 305), (71122, 305))

In [8]:
df.head()

,timestamp,track_id,conf_0,conf_1,conf_2,coord_x00,coord_y00,coord_x01,coord_y01,coord_x02,coord_y02,coord_x03,coord_y03,coord_x04,coord_y04,coord_x05,coord_y05,coord_x06,coord_y06,coord_x07,coord_y07,coord_x08,coord_y08,coord_x09,coord_y09,coord_x010,coord_y010,coord_x011,coord_y011,coord_x012,coord_y012,coord_x013,coord_y013,coord_x014,coord_y014,coord_x015,coord_y015,coord_x016,coord_y016,coord_x017,coord_y017,coord_x018,coord_y018,coord_x019,coord_y019,coord_x020,coord_y020,coord_x021,coord_y021,coord_x022,coord_y022,coord_x023,coord_y023,coord_x024,coord_y024,coord_x025,coord_y025,coord_x026,coord_y026,coord_x027,coord_y027,coord_x028,coord_y028,coord_x029,coord_y029,coord_x030,coord_y030,coord_x031,coord_y031,coord_x032,coord_y032,coord_x033,coord_y033,coord_x034,coord_y034,coord_x035,coord_y035,coord_x036,coord_y036,coord_x037,coord_y037,coord_x038,coord_y038,coord_x039,coord_y039,coord_x040,coord_y040,coord_x041,coord_y041,coord_x042,coord_y042,coord_x043,coord_y043,coord_x044,coord_y044,coord_x045,coord_y045,coord_x046,coord_y046,coord_x047,coord_y047,coord_x048,coord_y048,coord_x049,coord_y049,coord_x10,coord_y10,coord_x11,coord_y11,coord_x12,coord_y12,coord_x13,coord_y13,coord_x14,coord_y14,coord_x15,coord_y15,coord_x16,coord_y16,coord_x17,coord_y17,coord_x18,coord_y18,coord_x19,coord_y19,coord_x110,coord_y110,coord_x111,coord_y111,coord_x112,coord_y112,coord_x113,coord_y113,coord_x114,coord_y114,coord_x115,coord_y115,coord_x116,coord_y116,coord_x117,coord_y117,coord_x118,coord_y118,coord_x119,coord_y119,coord_x120,coord_y120,coord_x121,coord_y121,coord_x122,coord_y122,coord_x123,coord_y123,coord_x124,coord_y124,coord_x125,coord_y125,coord_x126,coord_y126,coord_x127,coord_y127,coord_x128,coord_y128,coord_x129,coord_y129,coord_x130,coord_y130,coord_x131,coord_y131,coord_x132,coord_y132,coord_x133,coord_y133,coord_x134,coord_y134,coord_x135,coord_y135,coord_x136,coord_y136,coord_x137,coord_y137,coord_x138,coord_y138,coord_x139,coord_y139,coord_x140,coord_y140,coord_x141,coord_y141,coord_x142,coord_y142,coord_x143,coord_y143,coord_x144,coord_y144,coord_x145,coord_y145,coord_x146,coord_y146,coord_x147,coord_y147,coord_x148,coord_y148,coord_x149,coord_y149,coord_x20,coord_y20,coord_x21,coord_y21,coord_x22,coord_y22,coord_x23,coord_y23,coord_x24,coord_y24,coord_x25,coord_y25,coord_x26,coord_y26,coord_x27,coord_y27,coord_x28,coord_y28,coord_x29,coord_y29,coord_x210,coord_y210,coord_x211,coord_y211,coord_x212,coord_y212,coord_x213,coord_y213,coord_x214,coord_y214,coord_x215,coord_y215,coord_x216,coord_y216,coord_x217,coord_y217,coord_x218,coord_y218,coord_x219,coord_y219,coord_x220,coord_y220,coord_x221,coord_y221,coord_x222,coord_y222,coord_x223,coord_y223,coord_x224,coord_y224,coord_x225,coord_y225,coord_x226,coord_y226,coord_x227,coord_y227,coord_x228,coord_y228,coord_x229,coord_y229,coord_x230,coord_y230,coord_x231,coord_y231,coord_x232,coord_y232,coord_x233,coord_y233,coord_x234,coord_y234,coord_x235,coord_y235,coord_x236,coord_y236,coord_x237,coord_y237,coord_x238,coord_y238,coord_x239,coord_y239,coord_x240,coord_y240,coord_x241,coord_y241,coord_x242,coord_y242,coord_x243,coord_y243,coord_x244,coord_y244,coord_x245,coord_y245,coord_x246,coord_y246,coord_x247,coord_y247,coord_x248,coord_y248,coord_x249,coord_y249
0,1578606007801600134,2,0.506040,0.381514,0.112446,-0.126374,0.190396,-0.218176,0.368930,-0.329786,0.611086,-0.437752,0.801056,-0.536092,0.976636,-0.615650,1.167148,-0.731062,1.340928,-0.842604,1.503320,-0.922524,1.653328,-1.017680,1.837852,-1.100672,1.952406,-1.168912,2.111148,-1.253038,2.250384,-1.339874,2.358542,-1.402512,2.482002,-1.467298,2.611506,-1.534434,2.730776,-1.594094,2.835644,-1.647480,2.955026,-1.699088,3.057472,-1.750400,3.152686,-1.803948,3.250502,-1.856828,3.342490,-1.911648,3.430850,-1.941562,3.517020,-1.989890,3.598168,-2.022126,3.668232,-2.068720,3.750476,-2.111012,3.824976,-2.146714,3.910620,-2.176090,3.980626,-2.208330,4.037934,-2.244488,4.120830,-2.275400,4.182750,-2.318258,4.258310,-2.

In [9]:
df.to_csv("submission.csv", index=False, float_format='%.6f')